In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem
import tuto

# Weather data and solar radiation on a tilted surface,

Melchior CHAILLOUX ; Ulysse UYTTERHAEGEN ; Arthur HEYVANG ; Léa LORA-RUNCO  
  
30/05/2023

## Introduction,

### Objectifs,

- Dessiner les plans d'un appartement
- Choisir les hypothèses pour créer un modèle
- Créer un modèle
- Faire fonctionner le modèle
- Intégrer les conditions liées à la météo



## Description de l'appartement

![schema_de_l'appartement](./Schemas_suite.svg)  
Schéma de l'appartement  

![schema_de_l'appartement_suite](./Schemas.svg)  
Schéma complet de l'appartement avec les échanges thermiques  

The dimensions and surface areas of the building are:
- $H=3 \: \mathrm{m}$ - height of the building;
- $L=10 \: \mathrm{m}$ - lengh of the building;
- $l= \frac{L}{2}$ - width of the building and lengh of each zone;
- $S_g= l \times H$   - surface area of the glass window;
- $S_c = S_i = 5 \times S_g$   - surface area of the 5 (concrete, insulation and plaster) walls.



In [8]:
H = 3               # m height of the building
L = 10              # m lengh of the building
l = L/2              # m width of the building
Sg = l*H         # m² surface of the glass wall
Sc = Si = Sp = 5 * Sg    # m² surface of concrete & insulation & plaster of the 5 walls

## Description des murs

![murs_et_vitres](./mur_et_vitre.svg)  
Vu en coupe des murs et des vitres  
  
Le plâtre est le BA13 dont les propriétés thermophysiques ont été trouvées sur [ce site](http://pigo.free.fr/_media/re-caracteristiques-thermiques.pdf). Le plâtre résiste à l'humidité et agit comme un isolant thermique.

In [9]:
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
T_ext = 0       # °C
σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

concrete = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Thickness': 0.2,
            'Surface': Sc}

insulation = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Thickness': 0.08,
              'Surface': Si}

plaster = {'Conductivity': 0.25,
           'Density': 825, 
           'Specific heat': 1008,
           'Thickness': 0.02,
           'Surface': Sp}

window = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Thickness': 0.04,
         'Surface': Sg}

wall = pd.DataFrame.from_dict({'Insulation': insulation,
                               'Concrete': concrete,
                               'Plaster': plaster,
                               'Window': window},
                              orient='index')
wall

,Conductivity,Density,Specific heat,Thickness,Surface
Insulation,0.027,55.0,1210,0.08,75.0
Concrete,1.400,2300.0,880,0.20,75.0
Plaster,0.250,825.0,1008,0.02,75.0
Window,1.400,2500.0,1210,0.04,15.0


Grâce à ces données, nous trouvons les conductances et les capacités, dont les formes générales sont :
- Conduction : $G_{cd}= \frac{\lambda}{e}S $ , avec e l'épaisseur
- Convection : $G_{cv}= hS $
- Radiatif : $G_{LW} = \frac{1}{1/G_1 + 1/G_{1,2} + 1/G_2}$,  
  
avec $G_{1} = 4 \sigma \bar{T}^3 \frac{\varepsilon_w}{1 - \varepsilon_w} S_1$ ; $G_{1,2} = 4 \sigma \bar{T}^3 F_{1,2} S_1 = 4 \sigma \bar{T}^3 F_{2,1} S_2$ ; $G_{2} = 4 \sigma \bar{T}^3 \frac{\varepsilon_g}{1 - \varepsilon_g} S_2$ ; $\bar{T}= 293 \N : \mathrm{K}$ la température moyenne, $F$ le facteur de vue ;  

In [10]:
# conduction
G_cd = wall['Conductivity'] / wall['Thickness'] * wall['Surface']
# convection
Gw = h * wall['Surface'][0]     # wall
Gg = h * wall['Surface'][2]     # glass
# view factor wall-glass
Fwg = window['Surface'] / concrete['Surface']
# long wave radiation
Tm = 20 + 273   # K, mean temp for radiative exchange

GLW1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * wall['Surface']['Concrete']
GLW12 = 4 * σ * Tm**3 * Fwg * wall['Surface']['Concrete']
GLW2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * wall['Surface']['Window']

GLW = 1 / (1 / GLW1 + 1 / GLW12 + 1 / GLW2)

## Les hypothèses simplificatrices
  
Pour simplifier le modèle en vue d'en améliorer sa compréhension, nous avons négligé plusieurs choses, les principales :  
- les coins des murs
- les longwaves radiations


## Equivalence de l'analyse thermique avec un schéma électrique  
![shema_resistance](./schema_resistance.svg)
Schéma de l'équivalence thermique/électrique  
  
La conduction thermique dans les solides est semblable à celle de l'électricité dans les conducteurs électriques. Dans un conducteur, le flux d'électricité est entraîné par une différence potentielle et le flux de chaleur l'est par la différence dans la température. En conduction électrique, la charge électrique est transportée d'un point d'un conducteur à un autre par le mouvement des électrons ; dans la conduction thermique, la chaleur est transportée d'un point du solide à un autre par la vibration des molécules du solide suite à l'augmentation d'énergie.  
  
  

## Les matrices locales pour les différents équivalences

![mur_et_porte](./porte_mur_milieu.svg)
  
  
Nous avons pour cela la matrice suivante pour l'équivalence du mur centrale avec la porte :

![mur_et_porte](./porte_mur_milieu.svg)
  
  
Nous avons pour cela la matrice suivante pour l'équivalence du mur centrale avec la porte :